In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# #Making the current directory as the directory where all the files are.
# % cd ./drive/"My Drive"/hw4

In [3]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.utils as utils
import pickle as pk
from torch.utils.data import DataLoader, Dataset 
import time
from torch.nn.utils.rnn import *
import torch.nn as nn
from torch.autograd import Variable

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
torch.cuda.empty_cache()

In [5]:
speech_train = np.load('train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load('dev_new.npy', allow_pickle=True, encoding='bytes')
speech_test = np.load('test_new.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load('./train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load('./dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")

Data Loading Sucessful.....


In [6]:
letter_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',\
             'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-', "'", '.', '_', '+', ' ','<sos>','<eos>']

In [7]:
letter_dict = {}
k = 1
for l in letter_list:
    letter_dict[l] = k
    k += 1

In [8]:
def transform_letter_to_index(text, letter_dict):
    
    '''
    :param transcript :(N, ) Transcripts are the text input
    :param letter_list: Letter list defined above
    :return letter_to_index_list: Returns a list for all the transcript sentence to index
    '''
    
    out = []

    for pos, line in enumerate(text):
        
        if len(line) > 0:
            line = line.astype(str)            
            chrs = " ".join(o for o in line)
            A = [letter_dict[c] for c in chrs]
            A.insert(0, letter_dict['<sos>'])
            A.append(letter_dict['<eos>'])
            out.append(np.array(A))

    return out

In [9]:
character_text_train = transform_letter_to_index(transcript_train, letter_dict)
character_text_valid = transform_letter_to_index(transcript_valid, letter_dict)
print("Transformed data sucessfully.....")

Transformed data sucessfully.....


In [10]:
np.shape(character_text_train)
character_text_train[0]

array([33, 20,  8,  5, 32,  6,  5, 13,  1, 12,  5, 32, 16, 18, 15,  4, 21,
        3,  5, 19, 32,  1, 32, 12,  9, 20, 20,  5, 18, 32, 15,  6, 32, 20,
       23, 15, 32, 20, 15, 32,  6, 15, 21, 18, 32, 25, 15, 21, 14,  7, 32,
        9, 14, 32, 14, 15, 22,  5, 13,  2,  5, 18, 32,  1, 14,  4, 32,  4,
        5,  3,  5, 13,  2,  5, 18, 34])

# DataLoader

In [11]:
class Speech2Text_Dataset(Dataset):
    def __init__(self, speech, text=None, train=True):
        self.speech = speech
        self.train = train
        
        if(text is not None):
            self.text = text

    def __len__(self):
        return self.speech.shape[0]

    def __getitem__(self, index):
        
        if(self.train):
            return torch.tensor(self.speech[index].astype(np.float32)), torch.tensor(self.text[index])
        else:
            return torch.tensor(self.speech[index].astype(np.float32))


In [12]:
def collate_train(batch_data):
    
    Frame = []
    Text = []
    frame_len = []
    text_len = []

    for i, (frames, text) in enumerate(batch_data):
        Frame.append(frames)
        Text.append(text)
        frame_len.append(frames.size(0))
        text_len.append(text.size(0))

    all_frames = pad_sequence(Frame, batch_first = False)
    all_text = pad_sequence(Text, batch_first = True)

    return all_frames, frame_len, all_text, text_len


def collate_test(batch_data):
    
    Frame = []
    frame_len = []

    for i, (frames) in enumerate(batch_data):
        Frame.append(frames)
        frame_len.append(frames.size(0))

    all_frames = pad_sequence(Frame, batch_first = False)

    return all_frames, frame_len

In [13]:
Speech2Text_train_Dataset = Speech2Text_Dataset(speech = speech_train, text = character_text_train)
Speech2Text_valid_Dataset = Speech2Text_Dataset(speech = speech_valid, text = character_text_valid)
Speech2Text_test_Dataset = Speech2Text_Dataset(speech = speech_test, text = None, train = False)

In [14]:
# data loader
train_loader = torch.utils.data.DataLoader(
        Speech2Text_train_Dataset,
        batch_size=64, shuffle=True, collate_fn=collate_train)

dev_loader = torch.utils.data.DataLoader(
        Speech2Text_valid_Dataset,
        batch_size=64, shuffle= True, collate_fn=collate_train)

test_loader = torch.utils.data.DataLoader(
        Speech2Text_test_Dataset,
        batch_size=1, shuffle= False, collate_fn=collate_test)


In [15]:
# k = 0

# for batch, (frames, seq_sizes, _, _) in enumerate(dev_loader):
#     if(k < 5):
#         print(frames.shape)
# #         print(seq_sizes)
#         k += 1

In [16]:
# dropouts

class LockedDrop(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, dropout=0.5):
        if not self.training or not dropout:
            return x
        m = x.data.new(1, x.size(1), x.size(2)).bernoulli_(1 - dropout)
        mask = Variable(m, requires_grad=False) / (1 - dropout)
        mask = mask.expand_as(x)
        return mask * x


# Pyramidal BLSTM

In [28]:
class pBLSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=1, bidirectional=True)
        self.drop = LockedDrop()

    def forward(self, x):
        '''
        :param x :(N,T) input to the pBLSTM
        :return output: (N,T,H) encoded sequence from pyramidal Bi-LSTM 
        '''
        # x is output of lstm layer and is a packed sequence
        
        pad, lengths = pad_packed_sequence(x, batch_first=True)
        if pad.size(1) % 2 > 0:
            # if odd, remove last frame
            pad = pad[:, :-1, :]
            
        h, k, l = np.shape(pad)
        pad = pad.view(h, k//2, l*2)

        new_len = lengths//2 
        
        x = self.drop.forward(pad, dropout = 0.15)
        
        input_x = pack_padded_sequence(x, new_len, enforce_sorted = False, batch_first=True)

        output, hidden = self.blstm(input_x)
        
        return output, hidden


# Encoder

In [29]:
class Encoder(nn.Module):

    def __init__(self, input_dim, hidden_dim, value_size=128,key_size=128):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_size=input_dim,hidden_size=hidden_dim,num_layers=1,bidirectional=True)
        
        # Here you need to define the blocks of pBLSTMs

        # output from previous lstm is 2*hidden_dim and after reshaping, it becomes 4*hidden_dim
        self.plstm1 = pBLSTM(4*hidden_dim, hidden_dim)
        self.plstm2 = pBLSTM(4*hidden_dim, hidden_dim)
        self.plstm3 = pBLSTM(4*hidden_dim, hidden_dim)

        self.key_network = nn.Linear(hidden_dim*2, value_size)
        self.value_network = nn.Linear(hidden_dim*2, key_size)

    def forward(self, x, lens):
        
         # x is already padded

        rnn_inp = pack_padded_sequence(x, lengths=lens, batch_first=False, enforce_sorted=False)
        outputs = self.lstm(rnn_inp)[0]
        
        # dimension of outputs is 2*hidden_dim and outputs is packed sequence
        outputs, _ = self.plstm1(outputs)
        outputs, _ = self.plstm2(outputs)
        outputs, _ = self.plstm3(outputs)

        linear_input, lens = pad_packed_sequence(outputs)
        
        keys = self.key_network(linear_input)
        value = self.value_network(linear_input)

        return keys, value, lens


# Attention

In [30]:
class Attention(nn.Module):
    
    def __init__(self):
        super(Attention, self).__init__()
    
    def forward(self, query, key, values, lengths):
        
        """
        :param query: (N, H), decoder state of a single timestep
        :param key: (N, T, K), key from encoder
        :param context: (N, T, H), encoded source sequences
        :param lengths: (N,), lengths of source sequences
        :returns: (N, H) attended source context, and (N, T) attention vectors
        """
        
        # print("ATTENTION!!!!")
        # print(np.shape(query), key.shape, value.shape)
        
        attention = torch.bmm(key, query.unsqueeze(2)).squeeze(2)
  
        mask = torch.arange(key.size(1)).unsqueeze(0) >= lengths.unsqueeze(1)
        
        attention.masked_fill_(mask.to(device), -1e9)
        attention = nn.functional.softmax(attention, dim=1)
        out = torch.bmm(attention.unsqueeze(1), values).squeeze(1)
        
        return out, attention


# Decoder

In [31]:
from torch.distributions.gumbel import Gumbel 
# np.random.rand??

In [32]:
# prediction = torch.tensor(np.random.rand(1, 35))
# print(prediction.shape)
# print(prediction)
# noise_prediction = Gumbel(prediction, torch.FloatTensor([0.1])).sample()
# print(noise_prediction)

# print(noise_prediction - prediction)

In [33]:
class Decoder(nn.Module):

    def __init__(self, vocab_size, hidden_dim, value_size=128, key_size=128,  isAttended=True):
        super(Decoder, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, hidden_dim)

        self.lstm1 = nn.LSTMCell(input_size=hidden_dim+value_size, hidden_size=hidden_dim)
        self.lstm2 = nn.LSTMCell(input_size=hidden_dim, hidden_size=key_size)

        self.isAttended = isAttended
        
        if(isAttended):
            self.attention = Attention()
        
        self.character_prob = nn.Linear(key_size+value_size, vocab_size)
            

    def forward(self, key, values, text=None, train=True, lens = None, gumbel_noise = 0.1, tf = 0.9):
        
        '''
        :param key :(T, N, key_size) Output of the Encoder Key projection layer
        :param values: (T, N, value_size) Output of the Encoder Value projection layer
        :param text: (N, text_len) Batch input of text with text_length
        :param train: Train or eval mode
        :lens: actual length for input text
        :return predictions: Returns the character perdiction probability 
        '''
        
        batch_size = key.shape[1]
        
        if(train):
            max_len =  text.shape[1] - 1
            embeddings = self.embedding(text)
            
        else:
            max_len = 250
            
        predictions = []

        hidden_states = [None, None]
        prediction = torch.zeros(batch_size, 1).to(device)
        
        # taking zero context for first character
#         context = torch.zeros(batch_size, 128).to(device)
#         gumble_noise = 0.4
        # print("max_len: ", max_len)
    
        context = values[0, :, :]

        for i in range(max_len):
            
            '''
            Here you should implement Gumble noise and teacher forcing techniques
            '''
                   
            choice = np.random.choice([1, 2], p = [tf, 1-tf])

            if(train):

                if (choice == 1):
                      char_embed = embeddings[:, i, :]

                else:
#                     noise_prediction = Gumbel(prediction.cpu(), torch.FloatTensor([gumbel_noise])).sample().to(device)                    
                    char_embed = self.embedding(prediction.argmax(dim=-1))

            else:
            # at testing time, simply take the max from last time step
                if(i==0):
                    # start from sos in test case
                    char_embed = self.embedding(prediction.argmax(dim=-1) + 33)
                else:
                    char_embed = self.embedding(prediction.argmax(dim=-1))
#                     noise_prediction = Gumbel(prediction.cpu(), torch.FloatTensor([gumbel_noise])).sample().to(device)
#                     char_embed = self.embedding(noise_prediction.argmax(dim=-1))

            # print(np.shape(char_embed), np.shape(context))
            inp = torch.cat([char_embed, context], dim=1)
            hidden_states[0] = self.lstm1(inp, hidden_states[0])

            inp_2 = hidden_states[0][0]
            hidden_states[1] = self.lstm2(inp_2, hidden_states[1])

            output = hidden_states[1][0]
            # print(output.shape)
            
            query = hidden_states[1][0]
#             query = query.unsqueeze(1)
#             context, _ = self.attention(query, key, values, lens)
            context, _ = self.attention(query, key.transpose(0, 1), values.transpose(0, 1), lens)

            prediction = self.character_prob(torch.cat([output, context], dim=1)) # batch_size, vocab_size
            
#             print("predictions: ", prediction.shape)
#             print(prediction.unsqueeze(1).shape)
            predictions.append(prediction.unsqueeze(1))
#             print("....", np.shape(predictions))
            
        return torch.cat(predictions, dim=1)


In [34]:
class Seq2Seq(nn.Module):

    def __init__(self,input_dim,vocab_size,hidden_dim,value_size=128, key_size=128,isAttended=True):
        super(Seq2Seq,self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim)
        self.decoder = Decoder(vocab_size, 2*hidden_dim)

    def forward(self,speech_input, speech_len, text_input=None,train=True, tf = 0.9):
        key, value, lens = self.encoder(speech_input, speech_len)
        
        if(train):
            predictions = self.decoder(key, value, text_input, lens = lens, tf = tf)
        else:
            predictions = self.decoder(key, value, text=None, train=False, lens = lens, tf = tf)
            
        return predictions


# Learning

In [35]:
model = Seq2Seq(input_dim=40, vocab_size=len(letter_list) + 1, hidden_dim= 256)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(reduce=None).to(device)

In [36]:
def train(model, train_loader, num_epochs, criterion, optimizer, dev_loader = None, valid = False):

        tf = 0.9
        k = 1
        for epochs in range(num_epochs):
            
            if(tf > 0.4 and k > 2):
                tf = 0.8*tf
                k = 1
            else:
                k += 1
                
            if(epochs > 20):
                optimizer = optim.Adam(model.parameters(), lr=0.0001)
#             if(epochs > 15):
#                 optimizer = optim.Adam(model.parameters(), lr=0.00001)
                
            start = time.time()
            
            print("tf: ", tf)
            print("epoch: ", epochs + 1)
            loss_sum_train = 0
            loss_sum_val = 0
            
            since = time.time()
            
            model.train()
            
            for (batch_num, collate_output) in enumerate(train_loader):
                
                with torch.autograd.set_detect_anomaly(True):

                    speech_input, speech_len, text_input, text_len = collate_output
                    speech_input = speech_input.to(device)
                    speech_len = torch.IntTensor(speech_len)
                    text_input = text_input.to(device)
                    text_len = torch.IntTensor(text_len)

                    predictions = model(speech_input, speech_len ,text_input, tf = tf) #, lens = speech_len)
                    mask = torch.zeros(text_input.size()).to(device)

                for i in range(len(text_len)):
                      mask[i, : text_len[i]] = 1

                mask = mask[:, 1:]
                mask = mask.reshape(-1).to(device)

                text_input2 = text_input[:, 1:]

                predictions = predictions.contiguous().view(-1, predictions.size(-1))
                text_input2 = text_input2.contiguous().view(-1)

                optimizer.zero_grad()

                loss = criterion(predictions, text_input2)
                masked_loss = torch.sum(loss*mask)

                
                batch_size = len(text_len)
                total_character = torch.sum(text_len) - batch_size
                
                ml = masked_loss/total_character
                ml.backward()

                torch.nn.utils.clip_grad_norm(model.parameters(), 2)
                optimizer.step()

                current_loss = float(masked_loss.item())/int(torch.sum(mask).item())

                if  batch_num % 150 == 1:
                      print('train_loss', batch_num, current_loss) 
                        
                loss_sum_train += current_loss
                
            print("train_loss: ", loss_sum_train/batch_num)
            
            if (valid and epochs% 2 == 0):
                
                model.eval()
                
                for (batch_num, collate_output) in enumerate(dev_loader):

                    speech_input, speech_len, text_input, text_len = collate_output
                    speech_input = speech_input.to(device)
                    speech_len = torch.IntTensor(speech_len)
                    text_input = text_input.to(device)
                    text_len = torch.IntTensor(text_len)

                    predictions = model(speech_input, speech_len ,text_input, train = False, tf = tf) #, lens = speech_len)
                    mask = torch.zeros(text_input.size()).to(device)

                    for i in range(len(text_len)):
                          mask[i, : text_len[i]] = 1

                    mask = mask[:, 1:]
                    mask = mask.reshape(-1).to(device)

                    text_input2 = text_input[:, 1:]
                    
                    max_len = text_input2.shape[1]
                    predictions = (predictions[:, :max_len, :]).contiguous().view(-1, predictions.size(-1))
                    text_input2 = text_input2.contiguous().view(-1)

                    loss = criterion(predictions, text_input2)
                    masked_loss = torch.sum(loss*mask)


                    batch_size = len(text_len)
                    total_character = torch.sum(text_len) - batch_size

                    ml = masked_loss/total_character

                    current_loss = float(masked_loss.item())/int(torch.sum(mask).item())

                    if  batch_num % 25 == 1:
                          print('valid_loss', batch_num, current_loss) 

                    loss_sum_val += current_loss
                
            print("valid_loss: ", loss_sum_val/batch_num)
            
            print("Time: ", time.time() - start)
            torch.save(model.state_dict(), "./run3 " + str(1 + epochs) + ".pt")


In [ ]:
model.load_state_dict(torch.load('./run6 1.pt'))
model.to(device)

In [ ]:
train(model, train_loader, 30, criterion, optimizer) #, dev_loader = dev_loader, valid = True)

tf:  0.9
epoch:  1


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


train_loss 1 3.077131984544837
train_loss 151 1.1463791797576302
train_loss 301 0.8898979302803541
train_loss:  1.0823632081334413
valid_loss:  0.0
Time:  1151.8983986377716
tf:  0.9
epoch:  2
train_loss 1 0.9494495008283821
train_loss 151 0.8343695062164709
train_loss 301 0.817195171196269
train_loss:  0.7980491656319868
valid_loss:  0.0
Time:  1150.3820641040802
tf:  0.7200000000000001
epoch:  3
train_loss 1 0.8855178592301594
train_loss 151 0.9226877273447097
train_loss 301 0.8907323673665173
train_loss:  0.9636828756048126
valid_loss:  0.0
Time:  1147.0211157798767
tf:  0.7200000000000001
epoch:  4
train_loss 1 1.0551269896456619
train_loss 151 0.9301653832572795
train_loss 301 0.7365605446348108
train_loss:  0.9052309024068864
valid_loss:  0.0
Time:  1149.2550394535065
tf:  0.7200000000000001
epoch:  5
train_loss 1 0.7459604658792651


In [ ]:
model.load_state_dict(torch.load('./eun2 27.pt'))
model.to(device)

# Testing

In [ ]:
def test(model, test_loader):


    A = []

    for (batch_num, collate_output) in enumerate(test_loader):
            mul_pred = []
            print("num", len(A))
            with torch.autograd.set_detect_anomaly(True):

                speech_input, speech_len = collate_output
                speech_input = speech_input.to(device)
                speech_len = torch.IntTensor(speech_len)
                prediction = model(speech_input, speech_len, train = False) 
                
#                 for i in range(100):
#                     prediction = model(speech_input, speech_len, train = False) 
#                     mul_pred.append(prediction)
                    
#                 avg_pred = np.log(mul_pred)             
                
                predictions = prediction.argmax(dim=-1)
                A.append(predictions)
            break
    return A

In [ ]:
def find_len(prob):
    # 100, 250, 35
    # find the actual length of these 100 sentences
    A  = np.argmax(prob, axis = -1)
    lengths = np.zeros(len(A))
    
    for i in range(len(A)):
        lengths[i] = np.argwhere(A[i] == 0) - 1
    
    return length


In [ ]:
A = np.random.rand(2, 1, 35)
A1 = [10, 20]
np.divide(A/A1)


In [ ]:
def test_A(model, test_loader):


    A = []

    for (batch_num, collate_output) in enumerate(test_loader):
            
            print("num", len(A))
            with torch.autograd.set_detect_anomaly(True):

                speech_input, speech_len = collate_output
                speech_input = speech_input.to(device)
                speech_len = torch.IntTensor(speech_len)
                prob = 0 
                
                for i in range(20):
                    
                    prediction = model(speech_input, speech_len, train = False) 
                    pred = prediction.squeeze(0).cpu().detach().numpy()
                  
                    max_prob = np.amax(pred, axis = 1)                    
                    argmax_prob = np.argmax(pred, axis = 1)
                    length = np.argwhere(argmax_prob == 0)[0] - 1
                    log_prob = np.log(max_prob)                  
                    
                    avg = (np.sum(log_prob)/length)[0]
                    
                    if(avg > prob):
                        prob = avg
                        pred = argmax_prob
                        
#             print(np.shape(pred))
            A.append(pred)
#             print(len(A))
            

    return A


In [ ]:
dev_loader = torch.utils.data.DataLoader(
        Speech2Text_valid_Dataset,
        batch_size=1, shuffle= False, collate_fn=collate_train)


In [ ]:
y = test_A(model, test_loader)

In [ ]:
y

In [ ]:
y = test(model, test_loader)

In [ ]:
# np.shape(y)
# y[0][0].cpu().numpy()

In [ ]:
sen = []

for i in range(len(y)):
    ex = y[i][0]
    ex = ex.cpu().numpy()
    ans =[]
    
    for e in ex:        
        if(e != 0):
            ans.append(letter_list[e - 1])        
    chrs = "".join(o for o in ans[:-1])
    sen.append(chrs)
  

In [ ]:
sen

In [ ]:
# len(sen), len(ex)
# str(sen[0])

In [ ]:
import csv

In [ ]:
with open('submit.csv', 'w') as fin:
    k = 0
    fin.write("Id" + "," + "Predicted")
    fin.write("\n")
    for s in sen:
        fin.write(str(k) + ",")
        fin.write(s)
        fin.write("\n")
        k += 1

In [ ]:
for batch, (frames, seq_sizes, text, _) in enumerate(dev_loader):
    print(frames.shape)
    print(seq_sizes)
    print(text)
    break

In [ ]:
# y = test(model, test_loader)